## Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).
Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. В данных представленна именно эта количественная мера активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

In [105]:
import pandas as pd
import numpy as np
import scipy
import statsmodels
import statistics
from statsmodels.stats.weightstats import *
import statsmodels.stats.multitest as smm

In [8]:
def write_answer(file_name, answer):
    with open(file_name, 'w') as fout:
        fout.write(str(answer))

### Часть 1: применение t-критерия Стьюдента

Для начала дважды применим критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках.

Критерий применим:
* для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

Ответом в этой части определим количество статистически значимых отличий, которые будут найдены с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [9]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep=',', header=0)

In [10]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [7]:
data.shape

(72, 15750)

In [13]:
data['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [14]:
early_neoplasia = data[data['Diagnosis']== 'early neoplasia'].drop(['Patient_id', 'Diagnosis'], axis=1)
normal = data[data['Diagnosis'] == 'normal'].drop(['Patient_id', 'Diagnosis'], axis=1)
cancer = data[data['Diagnosis'] == 'cancer'].drop(['Patient_id', 'Diagnosis'], axis=1)
data_columns = data.columns.drop(['Patient_id', 'Diagnosis'])

In [25]:
print('Shapiro-Wilk normality test, W-statistic:')
for i in data_columns:
    print('\tNormal neoplasia \'' + col_name + '\': %f, p-value: %f' % stats.shapiro(normal[i]))
    print('\tEarly neoplasia \'' + col_name + '\': %f, p-value: %f' % stats.shapiro(early_neoplasia[i]))
    print('\tCancer  \'' + col_name + '\': %f, p-value: %f' % stats.shapiro(cancer[i]))

# Вывод: при уровне значимости 0.05 не все данные распределены нормально

Shapiro-Wilk normality test, W-statistic:
	Normal neoplasia 'EIF1AY': 0.895629, p-value: 0.017408
	Early neoplasia 'EIF1AY': 0.897122, p-value: 0.015890
	Cancer  'EIF1AY': 0.853136, p-value: 0.003103
	Normal neoplasia 'EIF1AY': 0.670455, p-value: 0.000004
	Early neoplasia 'EIF1AY': 0.946906, p-value: 0.213321
	Cancer  'EIF1AY': 0.904603, p-value: 0.031487
	Normal neoplasia 'EIF1AY': 0.724633, p-value: 0.000022
	Early neoplasia 'EIF1AY': 0.930378, p-value: 0.088673
	Cancer  'EIF1AY': 0.843589, p-value: 0.002086
	Normal neoplasia 'EIF1AY': 0.920615, p-value: 0.060266
	Early neoplasia 'EIF1AY': 0.972330, p-value: 0.704477
	Cancer  'EIF1AY': 0.890680, p-value: 0.016324
	Normal neoplasia 'EIF1AY': 0.897267, p-value: 0.018844
	Early neoplasia 'EIF1AY': 0.985678, p-value: 0.970186
	Cancer  'EIF1AY': 0.940256, p-value: 0.181999
	Normal neoplasia 'EIF1AY': 0.929567, p-value: 0.095339
	Early neoplasia 'EIF1AY': 0.970829, p-value: 0.666147
	Cancer  'EIF1AY': 0.881133, p-value: 0.010537
	Normal ne

	Normal neoplasia 'EIF1AY': 0.941563, p-value: 0.176738
	Early neoplasia 'EIF1AY': 0.968691, p-value: 0.612074
	Cancer  'EIF1AY': 0.957226, p-value: 0.409708
	Normal neoplasia 'EIF1AY': 0.861553, p-value: 0.003610
	Early neoplasia 'EIF1AY': 0.953816, p-value: 0.305057
	Cancer  'EIF1AY': 0.972871, p-value: 0.757274
	Normal neoplasia 'EIF1AY': 0.938671, p-value: 0.152356
	Early neoplasia 'EIF1AY': 0.946133, p-value: 0.204831
	Cancer  'EIF1AY': 0.959404, p-value: 0.451434
	Normal neoplasia 'EIF1AY': 0.966563, p-value: 0.583331
	Early neoplasia 'EIF1AY': 0.881874, p-value: 0.007556
	Cancer  'EIF1AY': 0.968708, p-value: 0.658095
	Normal neoplasia 'EIF1AY': 0.967170, p-value: 0.597860
	Early neoplasia 'EIF1AY': 0.986990, p-value: 0.981237
	Cancer  'EIF1AY': 0.950610, p-value: 0.301267
	Normal neoplasia 'EIF1AY': 0.734645, p-value: 0.000030
	Early neoplasia 'EIF1AY': 0.906213, p-value: 0.025112
	Cancer  'EIF1AY': 0.929382, p-value: 0.106055
	Normal neoplasia 'EIF1AY': 0.941859, p-value: 0.179

	Early neoplasia 'EIF1AY': 0.911879, p-value: 0.033576
	Cancer  'EIF1AY': 0.951507, p-value: 0.314377
	Normal neoplasia 'EIF1AY': 0.970431, p-value: 0.677633
	Early neoplasia 'EIF1AY': 0.969945, p-value: 0.643663
	Cancer  'EIF1AY': 0.914342, p-value: 0.050460
	Normal neoplasia 'EIF1AY': 0.912272, p-value: 0.039519
	Early neoplasia 'EIF1AY': 0.845703, p-value: 0.001461
	Cancer  'EIF1AY': 0.777262, p-value: 0.000169
	Normal neoplasia 'EIF1AY': 0.973431, p-value: 0.751514
	Early neoplasia 'EIF1AY': 0.960348, p-value: 0.421440
	Cancer  'EIF1AY': 0.953975, p-value: 0.352988
	Normal neoplasia 'EIF1AY': 0.893576, p-value: 0.015770
	Early neoplasia 'EIF1AY': 0.934380, p-value: 0.109733
	Cancer  'EIF1AY': 0.812041, p-value: 0.000601
	Normal neoplasia 'EIF1AY': 0.951467, p-value: 0.291421
	Early neoplasia 'EIF1AY': 0.894300, p-value: 0.013815
	Cancer  'EIF1AY': 0.883923, p-value: 0.011962
	Normal neoplasia 'EIF1AY': 0.895553, p-value: 0.017345
	Early neoplasia 'EIF1AY': 0.757219, p-value: 0.0000

	Normal neoplasia 'EIF1AY': 0.958145, p-value: 0.402301
	Early neoplasia 'EIF1AY': 0.989748, p-value: 0.994871
	Cancer  'EIF1AY': 0.967687, p-value: 0.633815
	Normal neoplasia 'EIF1AY': 0.988208, p-value: 0.990242
	Early neoplasia 'EIF1AY': 0.968323, p-value: 0.602886
	Cancer  'EIF1AY': 0.935897, p-value: 0.146642
	Normal neoplasia 'EIF1AY': 0.979025, p-value: 0.877231
	Early neoplasia 'EIF1AY': 0.977475, p-value: 0.830943
	Cancer  'EIF1AY': 0.967290, p-value: 0.624452
	Normal neoplasia 'EIF1AY': 0.902124, p-value: 0.023881
	Early neoplasia 'EIF1AY': 0.949714, p-value: 0.247027
	Cancer  'EIF1AY': 0.964326, p-value: 0.555991
	Normal neoplasia 'EIF1AY': 0.949621, p-value: 0.265877
	Early neoplasia 'EIF1AY': 0.945389, p-value: 0.196956
	Cancer  'EIF1AY': 0.948873, p-value: 0.277244
	Normal neoplasia 'EIF1AY': 0.958965, p-value: 0.417978
	Early neoplasia 'EIF1AY': 0.970347, p-value: 0.653872
	Cancer  'EIF1AY': 0.963743, p-value: 0.542926
	Normal neoplasia 'EIF1AY': 0.944252, p-value: 0.202

	Cancer  'EIF1AY': 0.871692, p-value: 0.006907
	Normal neoplasia 'EIF1AY': 0.934840, p-value: 0.125089
	Early neoplasia 'EIF1AY': 0.956575, p-value: 0.350495
	Cancer  'EIF1AY': 0.947764, p-value: 0.262816
	Normal neoplasia 'EIF1AY': 0.974232, p-value: 0.770864
	Early neoplasia 'EIF1AY': 0.915019, p-value: 0.039504
	Cancer  'EIF1AY': 0.774379, p-value: 0.000153
	Normal neoplasia 'EIF1AY': 0.959329, p-value: 0.425097
	Early neoplasia 'EIF1AY': 0.925253, p-value: 0.067566
	Cancer  'EIF1AY': 0.927573, p-value: 0.096934
	Normal neoplasia 'EIF1AY': 0.947466, p-value: 0.238614
	Early neoplasia 'EIF1AY': 0.943774, p-value: 0.180851
	Cancer  'EIF1AY': 0.942148, p-value: 0.199804
	Normal neoplasia 'EIF1AY': 0.883807, p-value: 0.009922
	Early neoplasia 'EIF1AY': 0.911264, p-value: 0.032529
	Cancer  'EIF1AY': 0.973031, p-value: 0.761038
	Normal neoplasia 'EIF1AY': 0.964393, p-value: 0.532765
	Early neoplasia 'EIF1AY': 0.887734, p-value: 0.010018
	Cancer  'EIF1AY': 0.922932, p-value: 0.077004
	Norm

...

	Cancer  'EIF1AY': 0.844815, p-value: 0.002194
	Normal neoplasia 'EIF1AY': 0.950611, p-value: 0.279311
	Early neoplasia 'EIF1AY': 0.922207, p-value: 0.057534
	Cancer  'EIF1AY': 0.964201, p-value: 0.553171
	Normal neoplasia 'EIF1AY': 0.955053, p-value: 0.347256
	Early neoplasia 'EIF1AY': 0.961445, p-value: 0.444003
	Cancer  'EIF1AY': 0.989396, p-value: 0.995444
	Normal neoplasia 'EIF1AY': 0.955150, p-value: 0.348892
	Early neoplasia 'EIF1AY': 0.917904, p-value: 0.045910
	Cancer  'EIF1AY': 0.948023, p-value: 0.266124
	Normal neoplasia 'EIF1AY': 0.971003, p-value: 0.691791
	Early neoplasia 'EIF1AY': 0.928038, p-value: 0.078308
	Cancer  'EIF1AY': 0.971000, p-value: 0.712921
	Normal neoplasia 'EIF1AY': 0.897302, p-value: 0.018876
	Early neoplasia 'EIF1AY': 0.974174, p-value: 0.751218
	Cancer  'EIF1AY': 0.977216, p-value: 0.853726
	Normal neoplasia 'EIF1AY': 0.965778, p-value: 0.564785
	Early neoplasia 'EIF1AY': 0.989947, p-value: 0.995435
	Cancer  'EIF1AY': 0.966443, p-value: 0.604572
	Norm

	Early neoplasia 'EIF1AY': 0.912467, p-value: 0.034610
	Cancer  'EIF1AY': 0.971740, p-value: 0.730566
	Normal neoplasia 'EIF1AY': 0.958531, p-value: 0.409626
	Early neoplasia 'EIF1AY': 0.924096, p-value: 0.063558
	Cancer  'EIF1AY': 0.952240, p-value: 0.325445
	Normal neoplasia 'EIF1AY': 0.920272, p-value: 0.059224
	Early neoplasia 'EIF1AY': 0.907494, p-value: 0.026808
	Cancer  'EIF1AY': 0.946868, p-value: 0.251668
	Normal neoplasia 'EIF1AY': 0.955252, p-value: 0.350613
	Early neoplasia 'EIF1AY': 0.966150, p-value: 0.549829
	Cancer  'EIF1AY': 0.973262, p-value: 0.766425
	Normal neoplasia 'EIF1AY': 0.981802, p-value: 0.926584
	Early neoplasia 'EIF1AY': 0.953409, p-value: 0.298796
	Cancer  'EIF1AY': 0.951606, p-value: 0.315857
	Normal neoplasia 'EIF1AY': 0.967156, p-value: 0.597512
	Early neoplasia 'EIF1AY': 0.960913, p-value: 0.432955
	Cancer  'EIF1AY': 0.865240, p-value: 0.005206
	Normal neoplasia 'EIF1AY': 0.922937, p-value: 0.067850
	Early neoplasia 'EIF1AY': 0.965191, p-value: 0.5271

	Normal neoplasia 'EIF1AY': 0.825575, p-value: 0.000795
	Early neoplasia 'EIF1AY': 0.786479, p-value: 0.000136
	Cancer  'EIF1AY': 0.923191, p-value: 0.077998
	Normal neoplasia 'EIF1AY': 0.973129, p-value: 0.744157
	Early neoplasia 'EIF1AY': 0.963657, p-value: 0.492010
	Cancer  'EIF1AY': 0.936175, p-value: 0.148679
	Normal neoplasia 'EIF1AY': 0.979940, p-value: 0.894748
	Early neoplasia 'EIF1AY': 0.953004, p-value: 0.292685
	Cancer  'EIF1AY': 0.967914, p-value: 0.639206
	Normal neoplasia 'EIF1AY': 0.942820, p-value: 0.188474
	Early neoplasia 'EIF1AY': 0.833791, p-value: 0.000881
	Cancer  'EIF1AY': 0.948362, p-value: 0.270512
	Normal neoplasia 'EIF1AY': 0.990747, p-value: 0.997673
	Early neoplasia 'EIF1AY': 0.943008, p-value: 0.173671
	Cancer  'EIF1AY': 0.968070, p-value: 0.642910
	Normal neoplasia 'EIF1AY': 0.981722, p-value: 0.925329
	Early neoplasia 'EIF1AY': 0.938677, p-value: 0.137966
	Cancer  'EIF1AY': 0.941594, p-value: 0.194428
	Normal neoplasia 'EIF1AY': 0.973487, p-value: 0.752

	Early neoplasia 'EIF1AY': 0.750594, p-value: 0.000038
	Cancer  'EIF1AY': 0.943710, p-value: 0.215741
	Normal neoplasia 'EIF1AY': 0.953341, p-value: 0.319554
	Early neoplasia 'EIF1AY': 0.929186, p-value: 0.083231
	Cancer  'EIF1AY': 0.977436, p-value: 0.858186
	Normal neoplasia 'EIF1AY': 0.978903, p-value: 0.874807
	Early neoplasia 'EIF1AY': 0.954513, p-value: 0.316024
	Cancer  'EIF1AY': 0.948626, p-value: 0.273979
	Normal neoplasia 'EIF1AY': 0.961948, p-value: 0.478806
	Early neoplasia 'EIF1AY': 0.955699, p-value: 0.335476
	Cancer  'EIF1AY': 0.980499, p-value: 0.914486
	Normal neoplasia 'EIF1AY': 0.986883, p-value: 0.982930
	Early neoplasia 'EIF1AY': 0.948426, p-value: 0.230999
	Cancer  'EIF1AY': 0.968808, p-value: 0.660484
	Normal neoplasia 'EIF1AY': 0.960732, p-value: 0.453307
	Early neoplasia 'EIF1AY': 0.976373, p-value: 0.805178
	Cancer  'EIF1AY': 0.965223, p-value: 0.576351
	Normal neoplasia 'EIF1AY': 0.864101, p-value: 0.004040
	Early neoplasia 'EIF1AY': 0.957670, p-value: 0.3700

	Normal neoplasia 'EIF1AY': 0.955745, p-value: 0.359024
	Early neoplasia 'EIF1AY': 0.934920, p-value: 0.112938
	Cancer  'EIF1AY': 0.964084, p-value: 0.550543
	Normal neoplasia 'EIF1AY': 0.883407, p-value: 0.009738
	Early neoplasia 'EIF1AY': 0.964391, p-value: 0.508648
	Cancer  'EIF1AY': 0.965688, p-value: 0.587036
	Normal neoplasia 'EIF1AY': 0.664483, p-value: 0.000004
	Early neoplasia 'EIF1AY': 0.751793, p-value: 0.000039
	Cancer  'EIF1AY': 0.819627, p-value: 0.000803
	Normal neoplasia 'EIF1AY': 0.958174, p-value: 0.402846
	Early neoplasia 'EIF1AY': 0.978375, p-value: 0.851119
	Cancer  'EIF1AY': 0.941621, p-value: 0.194681
	Normal neoplasia 'EIF1AY': 0.918301, p-value: 0.053578
	Early neoplasia 'EIF1AY': 0.903153, p-value: 0.021502
	Cancer  'EIF1AY': 0.903642, p-value: 0.030071
	Normal neoplasia 'EIF1AY': 0.884210, p-value: 0.010112
	Early neoplasia 'EIF1AY': 0.967052, p-value: 0.571604
	Cancer  'EIF1AY': 0.968981, p-value: 0.664621
	Normal neoplasia 'EIF1AY': 0.838722, p-value: 0.001

Проверим гипотезу о равенстве средних в двух независимых выборках. Уровень значимости alpha = 0.05
* H0: средние значения РНК в генах равны
* H1: средние не равны

In [42]:
def genes_diffs(right, left, columns):
    return [scipy.stats.ttest_ind(right[i], left[i], equal_var = False).pvalue for i in columns]

In [60]:
normalEarly_diff_genes = genes_diffs(normal, early_neoplasia, data_columns)
normalEarly_diff_genes_count = 0
for p in range(len(normalEarly_diff_genes)):
    if normalEarly_diff_genes[p] < 0.05:
        normalEarly_diff_genes_count += 1
print(normalEarly_diff_genes_count)
write_answer('normal_early.txt', normalEarly_diff_genes_count)

1575


In [59]:
earlyCancer_diff_genes = genes_diffs(early_neoplasia, cancer, data_columns)
earlyCancer_diff_genes_count = 0
for p in range(len(earlyCancer_diff_genes)):
    if earlyCancer_diff_genes[p] < 0.05:
        earlyCancer_diff_genes_count += 1
print(earlyCancer_diff_genes_count)
write_answer('early_cancer.txt', earlyCancer_diff_genes_count)

3490


Видим что для первых групп насчитывается 1575 значимых отличий. Во втором случае - 3490 генов, у которых p-value теста оказался меньше, чем уровень значимости.  

### Часть 2: поправка методом Холма

Теперь применим поправку Холма для получившихся двух наборов достигаемых уровней значимости из первой части. Чтобы избавиться от проблемы, связанной с множественной проверкой, применим поправку Бонферрони, то есть возьмем уровень значимости alpha = 0.05/2.

В качестве ответа к этой части определим количество значимых отличий в каждой группе после того, как произведем коррекцию Холма-Бонферрони.

In [162]:
def column_names(array_of_indexes):
    names = []
    for i in array_of_indexes:
        names.append(list(data)[2:][i])
    return names
    
def fold_change(sample1, sample2):
    mean1 = statistics.mean(sample1)
    mean2 = statistics.mean(sample2)
    if mean1 < mean2:
        return mean2/mean1
    else:
        return mean1/mean2

In [166]:
reject_n_e, pvalues_normalEarly_holm, alphac_n_e_sidak, alphac_n_e_bonf = smm.multipletests(
    normalEarly_diff_genes, method = 'holm')
column_indexes = np.where(pvalues_normalEarly_holm < 0.025)

In [167]:
counter = 0
for i in column_names(column_indexes[0]):
    if fold_change(data[data['Diagnosis'] == 'normal'][i], data[data['Diagnosis'] == 'early neoplasia'][i]) > 1.5:
        counter += 1
print(counter)
write_answer('fold1.txt', counter)

2


In [169]:
reject_n_e, pvalues_normalEarly_holm, alphac_n_e_sidak, alphac_n_e_bonf = smm.multipletests(
    earlyCancer_diff_genes, method = 'holm')
column_indexes1 = np.where(pvalues_normalEarly_holm < 0.025)

In [171]:
counter1 = 0
for i in column_names(column_indexes1[0]):
    if fold_change(data[data['Diagnosis'] == 'cancer'][i], data[data['Diagnosis'] == 'early neoplasia'][i]) > 1.5:
        counter1 += 1
print(counter1)
write_answer('fold2.txt', counter1)

77


Видим, что, применив поправку Холма, количество значимых отличий снизилось: в первой группе стало равно 2, во второй - 77. Причем это числа уже с учетом практической значимости - для каждого значимого изменения посчитали переменную fold change и выписали те изменения, значения которых больше, чем 1.5.

### Часть 3: поправка методом Бенджамини-Хохберга

В этом случае мы повторим действия второй части, но будем использовать уже метод Бенджамини-Хохберга.

В итоге получим количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга с учетом fold change. 

In [173]:
reject_n_e, pvalues_normalEarly_holm, alphac_n_e_sidak, alphac_n_e_bonf = smm.multipletests(
    normalEarly_diff_genes, method = 'fdr_bh')
column_indexes = np.where(pvalues_normalEarly_holm < 0.025)

counter2 = 0
for i in column_names(column_indexes[0]):
    if fold_change(data[data['Diagnosis'] == 'normal'][i], data[data['Diagnosis'] == 'early neoplasia'][i]) > 1.5:
        counter2 += 1
print(counter)
write_answer('fold3.txt', counter2)

4


In [178]:
reject_n_e, pvalues_normalEarly_holm, alphac_n_e_sidak, alphac_n_e_bonf = smm.multipletests(
    earlyCancer_diff_genes, method = 'fdr_bh')
column_indexes1 = np.where(pvalues_normalEarly_holm < 0.025)

counter3 = 0
for i in column_names(column_indexes1[0]):
    if fold_change(data[data['Diagnosis'] == 'cancer'][i], data[data['Diagnosis'] == 'early neoplasia'][i]) > 1.5:
        counter3 += 1
print(counter3)
write_answer('fold4.txt', counter3)

524


Как видим, в первой группе результат - 4, во второй результат равен 524. Стоит помнить, что данный метод коррекции допускает больше ошибок первого рода и имеет большую мощность, относитьльно метода Холма.
Резюмируя, в ходе проверке гипотез мы определили те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.